In [ ]:
import os
import cv2
import copy
import pandas as pd
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from google.colab.patches import cv2_imshow
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import jaccard_score
from collections import Counter
import random
import torch
from sklearn import svm
from skimage.morphology import disk
from sklearn.preprocessing import StandardScaler
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.functional import Tensor
from scipy.ndimage.filters import median_filter
import torch.optim as optim
import torchvision.transforms as transforms
import itertools
import torchvision.datasets as datasets
from scipy.stats import kurtosis, skew
import torchvision.transforms.functional as TF
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.activation import ReLU
from sklearn.metrics import classification_report, balanced_accuracy_score, cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import kurtosis, skew
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
import statistics
from statistics import mode
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
import pickle
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.optim as optim
from collections import Counter
from tqdm import tqdm
from google.colab import drive
import itertools
from pathlib import Path
from sklearn.utils import shuffle


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Color Histograms
rf_colorh= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorHistRF_model.sav', 'rb'))
gb_colorh= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorHistGBoost_model.sav', 'rb'))
svm_colorh= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorHistSVM_model.sav', 'rb'))

#Color statistics
rf_colors= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorStatRF_model.sav', 'rb'))
svm_colors= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorStatSVM_model.sav', 'rb'))
gb_colors= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/ColorStatGBoost_model.sav', 'rb'))

#GLCM texture
rf_glcm= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/GLCMRF_model.sav', 'rb'))
knn_glcm= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/GLCMKNN_model.sav', 'rb'))
gb_glcm= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/GLCMGBoost_model.sav', 'rb'))

#SIFT + BoW
rf_sift= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/SiftRF_model.sav', 'rb'))
svm_sift= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/SiftSVM_model.sav', 'rb'))
gb_sift= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/SiftGBoost_model.sav', 'rb'))

#LBP histograms
rf_lbph= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/LBPHRF_model.sav', 'rb'))
svm_lbph= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/LBPHSVM_model.sav', 'rb'))
gb_lbph= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Models/WP_Aug/LBPHGBoost_model.sav', 'rb'))

In [ ]:
X_train= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/X_train.p','rb'))
X_test= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/X_val.p','rb'))
y_train= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/y_train.p','rb'))
y_test= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/y_val.p','rb'))

#Features: color histogram
X_train_ch= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Train_HistogramColorFeatures291022.csv')
X_test_ch= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Test_HistogramColorFeatures291022.csv')
scaler_ch= StandardScaler().fit(X_train_ch)
X_train_ch_scaled= scaler_ch.transform(X_train_ch)
X_test_ch_scaled= scaler_ch.transform(X_test_ch)

#Features: color statistics
X_train_cs= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Train_ColorFeatures291022.csv')
X_test_cs= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Test_ColorFeatures291022.csv')
scaler_cs= StandardScaler().fit(X_train_cs)
X_train_cs_scaled= scaler_cs.transform(X_train_cs)
X_test_cs_scaled= scaler_cs.transform(X_test_cs)

#Features: glcm
X_train_glcm= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Train_TextureFeatures291022.csv')
X_test_glcm= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/Test_TextureFeatures291022.csv')
scaler_glcm= StandardScaler().fit(X_train_glcm)
X_train_glcm_scaled= scaler_glcm.transform(X_train_glcm)
X_test_glcm_scaled= scaler_glcm.transform(X_test_glcm)

#Features: LBP histograms
LBP_features1_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_features1_train_291022.csv')
LBP_features2_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_features2_train_291022.csv')
LBP_features3_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_LBP_features3_train_291022.csv')
LBP_histogram_features_train= pd.concat([LBP_features1_train, LBP_features2_train, LBP_features3_train], axis=1)
X_train_lbph= LBP_histogram_features_train.copy()
LBP_features1_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_features1_test_291022.csv')
LBP_features2_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_features2_test_291022.csv')
LBP_features3_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/ThreeClassProblem_Data/LBP_features3_test_291022.csv')
LBP_histogram_features_test= pd.concat([LBP_features1_test, LBP_features2_test, LBP_features3_test], axis=1)
X_test_lbph= LBP_histogram_features_test.copy()
scaler_lbph= StandardScaler().fit(X_train_lbph)
X_train_lbph_scaled= scaler_lbph.transform(X_train_lbph)
X_test_lbph_scaled= scaler_lbph.transform(X_test_lbph)

#Features: SIFT + BoW
train_features_sift = pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Data/Pickles/train_features_wp.p','rb'))
test_features_sift= pickle.load(open('/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem/Data/Pickles/val_features_wp.p','rb'))
X_train_sift= pd.DataFrame(train_features_sift)
X_test_sift= pd.DataFrame(test_features_sift)
scaler_sift= StandardScaler().fit(X_train_sift)
X_train_sift_scaled= scaler_sift.transform(X_train_sift)
X_test_sift_scaled= scaler_sift.transform(X_test_sift)

In [ ]:
print(X_train_ch_scaled.shape)
print(X_train_cs_scaled.shape)
print(X_train_glcm_scaled.shape)
print(X_train_lbph_scaled.shape)
print(X_train_sift_scaled.shape)

(8058, 192)
(8058, 30)
(8058, 10)
(8058, 54)
(8058, 200)


In [ ]:
# models_list= [rf_colorh, gb_colorh, svm_colorh, rf_colors, svm_colors, gb_colors, rf_glcm, knn_glcm, gb_glcm, rf_lbph, svm_lbph, gb_lbph]
# features_list= [X_train_ch_scaled, X_test_ch_scaled, X_train_cs_scaled, X_test_cs_scaled, X_train_glcm_scaled, X_test_glcm_scaled, X_train_lbph_scaled, X_test_lbph_scaled]
# test_features_list=[X_test_ch_scaled, X_test_cs_scaled, X_test_glcm_scaled, X_test_lbph_scaled]

In [ ]:
models_list= [rf_colorh, gb_colorh, svm_colorh, rf_colors, svm_colors, gb_colors, rf_glcm, knn_glcm, gb_glcm]
features_list= [X_train_ch_scaled, X_test_ch_scaled, X_train_cs_scaled, X_test_cs_scaled, X_train_glcm_scaled, X_test_glcm_scaled]
test_features_list=[X_test_ch_scaled, X_test_cs_scaled, X_test_glcm_scaled]

In [ ]:
#Models results for verification 

for i in range(len(models_list)):
  if i in range(0,3):
    print("Model number", i, "corresponding to", models_list[i])
    y_pred= models_list[i].predict(test_features_list[0])
    print(classification_report(y_test, y_pred))
    print(balanced_accuracy_score(y_test,y_pred))
    print("Kappa Score :")
    print(cohen_kappa_score(y_test,y_pred))

  elif i in range(3,6):
    print("Model number", i, "corresponding to", models_list[i])
    y_pred= models_list[i].predict(test_features_list[1])
    print(classification_report(y_test,y_pred))
    print(balanced_accuracy_score(y_test,y_pred))
    print("Kappa Score :")
    print(cohen_kappa_score(y_test,y_pred))
  
  elif i in range(6,9):
    print("Model number", i, "corresponding to", models_list[i])
    y_pred= models_list[i].predict(test_features_list[2])
    print(classification_report(y_test,y_pred))
    print(balanced_accuracy_score(y_test,y_pred))
    print("Kappa Score :")
    print(cohen_kappa_score(y_test,y_pred))

Model number 0 corresponding to RandomForestClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.90      0.85      0.87       678
           1       0.79      0.81      0.80       498
           2       0.45      0.57      0.50        94

    accuracy                           0.81      1270
   macro avg       0.71      0.74      0.73      1270
weighted avg       0.82      0.81      0.82      1270

0.7436152694629645
Kappa Score :
0.6678085190211444
Model number 1 corresponding to GradientBoostingClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.88      0.77      0.82       678
           1       0.73      0.70      0.71       498
           2       0.28      0.59      0.38        94

    accuracy                           0.73      1270
   macro avg       0.63      0.69      0.64      1270
weighted avg       0.77      0.73      0.75      1270

0.6859430441278073
Kappa Score :
0.542

Ensemble model 1

In [ ]:
model1= models_list[0]
model2= models_list[1]
model3= models_list[2]
pred_m1= model1.predict(X_test_ch_scaled)
pred_m2= model2.predict(X_test_ch_scaled)
pred_m3= model3.predict(X_test_ch_scaled)

In [ ]:
new_ypred1=[]
for i in range(len(y_test)):
  temp_pred=[pred_m1[i], pred_m2[i], pred_m3[i]]
  if temp_pred.count(0) > temp_pred.count(1):
    if temp_pred.count(0) > temp_pred.count(2):
      new_ypred1.append(0)
    elif temp_pred.count(2) > temp_pred.count(0):
      new_ypred1.append(0)
    else: 
      new_ypred1.append(2)
  elif temp_pred.count(1) > temp_pred.count(0):
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred1.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred1.append(2)
    else: 
      new_ypred1.append(1)
  else:
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred1.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred1.append(2)
    else: 
      new_ypred1.append(1)

In [ ]:
print(classification_report(y_test,new_ypred1))
print(balanced_accuracy_score(y_test,new_ypred1))
print("Kappa Score:")
print(cohen_kappa_score(y_test,new_ypred1))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85       678
           1       0.74      0.78      0.76       498
           2       0.40      0.55      0.46        94

    accuracy                           0.78      1270
   macro avg       0.68      0.71      0.69      1270
weighted avg       0.80      0.78      0.79      1270

0.7138364542082218
Kappa Score:
0.6129913113637645


Ensemble model 2

In [ ]:
model4= models_list[3]
model5= models_list[4]
model6= models_list[5]
pred_m4= model4.predict(X_test_cs_scaled)
pred_m5= model5.predict(X_test_cs_scaled)
pred_m6= model6.predict(X_test_cs_scaled)

In [ ]:
new_ypred2=[]
for i in range(len(y_test)):
  temp_pred=[pred_m4[i], pred_m5[i], pred_m6[i]]
  if temp_pred.count(0) > temp_pred.count(1):
    if temp_pred.count(0) > temp_pred.count(2):
      new_ypred2.append(0)
    elif temp_pred.count(2) > temp_pred.count(0):
      new_ypred2.append(2)
    else: 
      new_ypred2.append(0)
  elif temp_pred.count(1) > temp_pred.count(0):
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred2.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred2.append(2)
    else: 
      new_ypred2.append(1)
  else:
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred2.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred2.append(2)
    else: 
      new_ypred2.append(1)

In [ ]:
print(classification_report(y_test,new_ypred2))
print(balanced_accuracy_score(y_test,new_ypred2))
print("Kappa Score:")
print(cohen_kappa_score(y_test,new_ypred2))

              precision    recall  f1-score   support

           0       0.85      0.77      0.81       678
           1       0.71      0.72      0.72       498
           2       0.29      0.48      0.36        94

    accuracy                           0.73      1270
   macro avg       0.62      0.66      0.63      1270
weighted avg       0.76      0.73      0.74      1270

0.656150743688997
Kappa Score:
0.5311319458754036


Ensemble model 3

In [ ]:
model7= models_list[6]
model8= models_list[7]
model9= models_list[8]
pred_m7= model7.predict(X_test_glcm_scaled)
pred_m8= model8.predict(X_test_glcm_scaled)
pred_m9= model9.predict(X_test_glcm_scaled)

In [ ]:
new_ypred3=[]
for i in range(len(y_test)):
  temp_pred=[pred_m7[i], pred_m8[i], pred_m9[i]]
  if temp_pred.count(0) > temp_pred.count(1):
    if temp_pred.count(0) > temp_pred.count(2):
      new_ypred3.append(0)
    elif temp_pred.count(2) > temp_pred.count(0):
      new_ypred3.append(2)
    else: 
      new_ypred3.append(0)
  elif temp_pred.count(1) > temp_pred.count(0):
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred3.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred3.append(2)
    else: 
      new_ypred3.append(1)
  else:
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred3.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred3.append(2)
    else: 
      new_ypred3.append(1)

In [ ]:
print(classification_report(y_test,new_ypred3))
print(balanced_accuracy_score(y_test,new_ypred3))
print("Kappa Score:")
print(cohen_kappa_score(y_test,new_ypred3))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       678
           1       0.65      0.72      0.68       498
           2       0.23      0.23      0.23        94

    accuracy                           0.68      1270
   macro avg       0.55      0.55      0.55      1270
weighted avg       0.68      0.68      0.68      1270

0.5536275523231443
Kappa Score:
0.425679103852042


Ensemble of ensemble

In [ ]:
new_ypred7=[]
for i in range(len(y_test)):
  temp_pred=[new_ypred1[i], new_ypred2[i], new_ypred3[i]]
  if temp_pred.count(0) > temp_pred.count(1):
    if temp_pred.count(0) > temp_pred.count(2):
      new_ypred7.append(0)
    elif temp_pred.count(2) > temp_pred.count(0):
      new_ypred7.append(2)
    else: 
      new_ypred7.append(0)
  elif temp_pred.count(1) > temp_pred.count(0):
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred7.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred7.append(2)
    else: 
      new_ypred7.append(1)
  else:
    if temp_pred.count(1) > temp_pred.count(2):
      new_ypred7.append(1)
    elif temp_pred.count(2) > temp_pred.count(1):
      new_ypred7.append(2)
    else: 
      new_ypred7.append(1)

In [ ]:
print(classification_report(y_test,new_ypred7))
print(balanced_accuracy_score(y_test,new_ypred7))
print("Kappa Score:")
print(cohen_kappa_score(y_test,new_ypred7))

              precision    recall  f1-score   support

           0       0.87      0.80      0.84       678
           1       0.73      0.80      0.76       498
           2       0.41      0.45      0.43        94

    accuracy                           0.77      1270
   macro avg       0.67      0.68      0.67      1270
weighted avg       0.78      0.77      0.78      1270

0.681805109095139
Kappa Score:
0.5982654769279435


In [ ]:
model1= rf_colorh
model2= rf_colors
model3= rf_glcm

In [ ]:
pred_m1= model1.predict(X_test_ch_scaled)
pred_m2= model2.predict(X_test_cs_scaled)
pred_m3= model3.predict(X_test_glcm_scaled)

In [ ]:
final_pred=(pred_m1+pred_m2+pred_m3)/3.0

In [ ]:
final_pred_round=[]
for i in final_pred:
  if i >= 1.4:
    final_pred_round.append(2)
  elif i <= 0.7:
    final_pred_round.append(0)
  else:
    final_pred_round.append(1)

In [ ]:
print(classification_report(y_test,final_pred_round))
print(balanced_accuracy_score(y_test,final_pred_round))
print("Kappa Score:")
print(cohen_kappa_score(y_test,final_pred_round))

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       678
           1       0.77      0.65      0.71       498
           2       0.54      0.37      0.44        94

    accuracy                           0.76      1270
   macro avg       0.70      0.64      0.66      1270
weighted avg       0.76      0.76      0.76      1270

0.642043602767311
Kappa Score:
0.5614823197912095


In [ ]:
final_model = VotingClassifier(
    estimators=[('rf1', model1), ('rf2', model2), ('rf3', model3)], voting='hard')
final_model.fit(X_train_ch_scaled,y_train)
pred_final= final_model.predict(X_test_ch_scaled)
print(classification_report(y_test,pred_final))
print(balanced_accuracy_score(y_test,pred_final))
print("Kappa Score:")
print(cohen_kappa_score(y_test,pred_final))

              precision    recall  f1-score   support

           0       0.90      0.85      0.87       678
           1       0.79      0.81      0.80       498
           2       0.45      0.57      0.50        94

    accuracy                           0.81      1270
   macro avg       0.71      0.74      0.73      1270
weighted avg       0.82      0.81      0.82      1270

0.7436152694629645
Kappa Score:
0.6678085190211444
